## Final Project

### Inspecting transfusion.data file

In [18]:
import pandas as pd



### Loading the blood donations data

In [19]:
transfusion = pd.read_csv('transfusion.data')

In [20]:
transfusion.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


### Inspecting the transfusion dataframe

In [21]:
transfusion.describe()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
count,748.000000,748.000000,748.000000,748.000000,748.000000
mean,9.506684,5.514706,1378.676471,34.282086,0.237968
std,8.095396,5.839307,1459.826781,24.376714,0.426124
min,0.000000,1.000000,250.000000,2.000000,0.000000
25%,2.750000,2.000000,500.000000,16.000000,0.000000
50%,7.000000,4.000000,1000.000000,28.000000,0.000000
75%,14.000000,7.000000,1750.000000,50.000000,0.000000
max,74.000000,50.000000,12500.000000,98.000000,1.000000


In [22]:
transfusion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 5 columns):
Recency (months)                              748 non-null int64
Frequency (times)                             748 non-null int64
Monetary (c.c. blood)                         748 non-null int64
Time (months)                                 748 non-null int64
whether he/she donated blood in March 2007    748 non-null int64
dtypes: int64(5)
memory usage: 29.3 KB


### Creating target column

In [23]:
transfusion.rename(
    columns = {'whether he/she donated blood in March 2007': 'target'},
    inplace = True
)

In [24]:
transfusion.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),target
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


### Checking target incidence

In [25]:
transfusion.target.value_counts(normalize = True)

0    0.762032
1    0.237968
Name: target, dtype: float64

### Splitting transfusion into train and test datasets

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
transfusion.drop(columns = 'target'),
transfusion.target,
test_size = 0.25,
random_state = 42,
stratify = transfusion.target)

In [27]:
X_train.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
334,16,2,500,16
99,5,7,1750,26
116,2,7,1750,46
661,16,2,500,16
154,2,1,250,2


### Selecting the model using Tpot

In [28]:
pip install tpot


The following command must be run outside of the IPython shell:

    $ pip install tpot

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [29]:
from tpot import TPOTClassifier
from sklearn.metrics import roc_auc_score

ModuleNotFoundError: No module named 'tpot'

In [30]:
#instantiate classifier
tpot = TPOTClassifier(
generations = 5,
population_size=20,
verbosity = 2,
scoring = 'roc_auc',
random_state = 42,
disable_update_check = True,
config_dict = 'TPOT light')

NameError: name 'TPOTClassifier' is not defined

In [31]:
tpot.fit(X_train, y_train)

NameError: name 'tpot' is not defined

In [32]:
#AUC score for tpot model
tpot_auc_score = roc_auc_score(y_test, tpot.predict_proba(X_test)[:,1])
print(f'\nAUC score: {tpot_auc_score:.4f}')

NameError: name 'roc_auc_score' is not defined

In [33]:
#pipeline steps
print('\nBest pipeline steps:', end ='\n')
for idx, (name, transform) in enumerate(tpot.fitted_pipeline_.steps, start=1):
    print(f'{idx}.{transform}')


Best pipeline steps:


NameError: name 'tpot' is not defined

### Checking the Variance

In [34]:
X_train.var()

Recency (months)         6.692902e+01
Frequency (times)        3.382982e+01
Monetary (c.c. blood)    2.114364e+06
Time (months)            6.111466e+02
dtype: float64

### Log Normalization

In [35]:
import numpy as np

In [36]:
X_train_norm = X_train.copy()
X_test_norm = X_test.copy()

In [37]:
norm_col = 'Monetary (c.c. blood)'

In [38]:
for df_ in [X_train_norm, X_test_norm]:
    #add log normalized column
    df_['monetary_log'] = np.log(df_[norm_col])
    
    #drop og column
    df_.drop(columns = norm_col, inplace = True)

In [39]:
X_train_norm.var().round(3)

Recency (months)      66.929
Frequency (times)     33.830
Time (months)        611.147
monetary_log           0.837
dtype: float64

### Training the linear regression model

In [40]:
from sklearn.linear_model import LogisticRegression

In [41]:
clf = LogisticRegression(
solver = 'liblinear',
random_state = 42)

In [42]:
#train
clf.fit(X_train_norm, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [43]:
#auc score
clf_score = roc_auc_score(y_test, clf.predict_proba(X_test_norm)[:,1])
print('AUC_score:', clf_score)

NameError: name 'roc_auc_score' is not defined

In [44]:
#compare models
from operator import itemgetter

sorted(
[('tpot', tpot_auc_score), ('clf', clf_score)],
key = itemgetter(1),
reverse=True)

NameError: name 'tpot_auc_score' is not defined

In [45]:
#saving model
import pickle 
pickle.dump(clf, open("saved_model.pkl", "wb"))